In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [3]:
import numpy as np
import pandas as pd

In [75]:
from sklearn.preprocessing import StandardScaler

In [155]:
from sklearn.decomposition import PCA

In [177]:
# Cross Validation
from sklearn.model_selection import KFold

In [6]:
df = pd.read_csv('dataSet03.csv')

In [9]:
df = df.drop(columns=['Unnamed: 0'])

In [12]:
data = df.set_index('index')

In [105]:
data

,매출액_b3,매출액_b2,매출액_b1,영업이익_b3,영업이익_b2,영업이익_b1,당기순이익_b3,당기순이익_b2,당기순이익_b1,영업이익률_b3,...,순이익률_b2,순이익률_b1,부채비율_b3,부채비율_b2,부채비율_b1,당좌비율_b3,당좌비율_b2,당좌비율_b1,rank_val,target
index,,,,,,,,,,,,,,,,,,,,,
0,8720.0,9819.0,12084.0,211.00,481.00,754.00,-33.00,767.00,88.00,2.42,...,7.81,0.73,451.43,273.80,309.37,29.75,46.83,32.53,18.0,1
1,253.0,394.0,408.0,-109.00,-40.00,-153.00,-423.00,236.00,64.00,-42.91,...,59.84,15.66,65.87,80.00,72.92,121.10,105.86,75.64,17.0,1
2,5918.0,5287.0,4866.0,463.00,643.00,905.00,250.00,572.00,822.00,7.82,...,10.81,16.90,168.76,123.83,102.86,91.30,164.07,129.68,17.0,1
3,1544.0,1502.0,4350.0,146.00,379.00,612.00,179.00,360.00,262.00,9.48,...,23.96,6.02,7.88,14.36,11.60,423.53,365.88,507.76,21.0,1
5,319991.0,344840.0,409249.0,13903.00,18818.00,21542.00,2040.00,8070.00,6868.00,4.35,...,2.34,1.68,159.97,154.84,172.03,73.36,69.78,63.95,22.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,-116.0,584.0,-4089.0,3.35,5.42,-11.70,-0.64,2.32,-14.20,-2.71,...,522.12,2631.81,2486.25,2853.75,360.75,-3641.00,18311.00,-128166.00,20.0,1
857,-874.0,-409.0,-511.0,1.54,-7.81,-0.76,-10.00,-6.36,-7.41,-24.36,...,157.12,212.41,2408.18,1606.47,1434.57,-3193.00,-1041.00,-873.00,16.0,1
858,826.0,35.0,-815.0,4.33,0.60,-8.12,8.98,0.32,-7.97,19.01,...,80.28,108.19,185.40,178.62,128.78,2382.00,106.00,-2336.00,19.0,1


In [106]:
temp = data.iloc[:,:-2]

In [1256]:
scaler = StandardScaler()

scaler.fit(temp)

scaled_X = scaler.transform(temp)

In [1257]:
X = scaled_X

y = data.iloc[:,-1]

In [1258]:
# 데이터 전처리 (예시: 텐서로 변환)
x = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [1259]:
# 데이터 분할 (예시: 훈련 세트와 테스트 세트로 분할)
train_size = int(0.8 * len(x))  # 훈련 세트의 크기 설정
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [1304]:
# 모델 구성
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 4*input_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*input_size, 2*input_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(2*input_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.sigmoid(out)
        return out

input_size = x.shape[1]  # 입력 특성의 크기
model = BinaryClassifier(input_size)


# 손실 함수와 옵티마이저 설정
criterion = nn.BCELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
num_epochs = 20
batch_size = 64

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 미니배치 학습
    for i in range(0, len(x_train), batch_size):
        batch_x, batch_y = x_train[i:i+batch_size], y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += accuracy_score(batch_y.detach().numpy(), (outputs > 0.5).float().detach().numpy())

    epoch_loss /= len(x_train) / batch_size
    epoch_acc /= len(x_train) / batch_size

    print(f'Epoch {epoch+1} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    predictions = (outputs > 0.5).float()
    test_acc = accuracy_score(y_test.detach().numpy(), predictions.detach().numpy())
    print(f'Test Accuracy: {test_acc:.4f}')

Epoch 1 - Loss: 0.7266 - Accuracy: 0.5596
Epoch 2 - Loss: 0.6147 - Accuracy: 0.8428
Epoch 3 - Loss: 0.6022 - Accuracy: 0.8156
Epoch 4 - Loss: 0.5762 - Accuracy: 0.8280
Epoch 5 - Loss: 0.5646 - Accuracy: 0.8473
Epoch 6 - Loss: 0.5597 - Accuracy: 0.8426
Epoch 7 - Loss: 0.5520 - Accuracy: 0.8450
Epoch 8 - Loss: 0.5451 - Accuracy: 0.8619
Epoch 9 - Loss: 0.5390 - Accuracy: 0.8717
Epoch 10 - Loss: 0.5326 - Accuracy: 0.8596
Epoch 11 - Loss: 0.5310 - Accuracy: 0.8664
Epoch 12 - Loss: 0.5263 - Accuracy: 0.8906
Epoch 13 - Loss: 0.5194 - Accuracy: 0.8934
Epoch 14 - Loss: 0.5231 - Accuracy: 0.8717
Epoch 15 - Loss: 0.5156 - Accuracy: 0.8786
Epoch 16 - Loss: 0.5143 - Accuracy: 0.8909
Epoch 17 - Loss: 0.5114 - Accuracy: 0.8837
Epoch 18 - Loss: 0.4950 - Accuracy: 0.8837
Epoch 19 - Loss: 0.4947 - Accuracy: 0.8786
Epoch 20 - Loss: 0.4913 - Accuracy: 0.9126
Test Accuracy: 0.7596


# 변수확장

In [1339]:
temp_df = data.iloc[:,:-2]

In [1340]:
cols = temp_df.columns

In [1341]:
# 제곱항

for col in cols:
    new_col_name = col + '^2'
    temp_df[new_col_name] = temp_df[col] ** 2

# 세제곱항

for col in cols:
    new_col_name = col + '^3'
    temp_df[new_col_name] = temp_df[col] ** 3

# 네제곱항

for col in cols:
    new_col_name = col + '^4'
    temp_df[new_col_name] = temp_df[col] ** 4

# 5제곱항

for col in cols:
    new_col_name = col + '^5'
    temp_df[new_col_name] = temp_df[col] ** 5

In [1342]:
temp_df

,매출액_b3,매출액_b2,매출액_b1,영업이익_b3,영업이익_b2,영업이익_b1,당기순이익_b3,당기순이익_b2,당기순이익_b1,영업이익률_b3,...,영업이익률_b1^5,순이익률_b3^5,순이익률_b2^5,순이익률_b1^5,부채비율_b3^5,부채비율_b2^5,부채비율_b1^5,당좌비율_b3^5,당좌비율_b2^5,당좌비율_b1^5
index,,,,,,,,,,,,,,,,,,,,,
0,8720.0,9819.0,12084.0,211.00,481.00,754.00,-33.00,767.00,88.00,2.42,...,9.460693e+03,-7.923517e-03,2.905729e+04,2.073072e-01,1.874788e+13,1.538747e+12,2.833942e+12,2.330423e+07,2.252272e+08,3.642674e+07
1,253.0,394.0,408.0,-109.00,-40.00,-153.00,-423.00,236.00,64.00,-42.91,...,-7.327208e+07,-1.295036e+11,7.672871e+08,9.418002e+05,1.240047e+09,3.276800e+09,2.061737e+09,2.604478e+10,1.329412e+10,2.476040e+09
2,5918.0,5287.0,4866.0,463.00,643.00,905.00,250.00,572.00,822.00,7.82,...,2.220225e+06,1.338327e+03,1.476143e+05,1.378585e+06,1.368824e+11,2.911584e+10,1.151417e+10,6.343864e+09,1.188902e+11,3.667457e+10
3,1544.0,1502.0,4350.0,146.00,379.00,612.00,179.00,360.00,262.00,9.48,...,5.514047e+05,2.082298e+05,7.896490e+06,7.906467e+03,3.038303e+04,6.106217e+05,2.100342e+05,1.362765e+13,6.556821e+12,3.375145e+13
5,319991.0,344840.0,409249.0,13903.00,18818.00,21542.00,2040.00,8070.00,6868.00,4.35,...,4.026509e+03,1.073742e-01,7.015834e+01,1.338278e+01,1.047593e+11,8.900529e+10,1.506680e+11,2.124695e+09,1.654454e+09,1.069554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,-116.0,584.0,-4089.0,3.35,5.42,-11.70,-0.64,2.32,-14.20,-2.71,...,-4.552539e+10,3.150395e+13,3.880178e+13,1.262620e+17,9.500008e+16,1.892691e+17,6.109866e+12,-6.398872e+17,2.058545e+21,-3.458312e+25
857,-874.0,-409.0,-511.0,1.54,-7.81,-0.76,-10.00,-6.36,-7.41,-24.36,...,-3.503609e+05,5.206556e+11,9.575409e+10,4.323892e+11,8.099249e+16,1.069949e+16,6.075873e+15,-3.318903e+17,-1.222513e+15,-5.070739e+14
858,826.0,35.0,-815.0,4.33,0.60,-8.12,8.98,0.32,-7.97,19.01,...,-2.476099e+06,4.049411e+09,3.334547e+09,1.482298e+10,2.190527e+11,1.818237e+11,3.541947e+10,7.668471e+16,1.338226e+10,-6.956074e+16


In [1343]:
# 스케일링

scaler = StandardScaler()

scaler.fit(temp_df)

scaled_X = scaler.transform(temp_df)

In [1344]:
scaled_X = pd.DataFrame(scaled_X)

In [1345]:
scaled_X

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,-0.014066,-0.045401,-0.041744,-0.061553,-0.074429,-0.029644,-0.107247,-0.024041,-0.097066,0.087398,...,0.063789,-0.135599,-0.083950,-0.047553,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586
1,-0.189707,-0.195570,-0.193058,-0.179525,-0.155668,-0.171554,-0.336254,-0.085766,-0.101433,-0.370819,...,0.062867,-0.135688,-0.083950,-0.047553,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586
2,-0.072191,-0.117610,-0.135285,0.031350,-0.049169,-0.006018,0.058930,-0.046708,0.036496,0.141984,...,0.063817,-0.135599,-0.083950,-0.047553,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586
3,-0.162926,-0.177916,-0.141972,-0.085516,-0.090334,-0.051861,0.017239,-0.071352,-0.065404,0.158764,...,0.063796,-0.135599,-0.083950,-0.047553,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586
4,6.442995,5.292476,5.105276,4.986164,2.784829,3.222878,1.110016,0.824887,1.136649,0.106908,...,0.063789,-0.135599,-0.083950,-0.047553,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,-0.197362,-0.192542,-0.251337,-0.138106,-0.148585,-0.149446,-0.088245,-0.112930,-0.115662,0.035542,...,-0.509257,-0.113877,-0.073826,-0.037985,-0.067132,-0.049540,-0.049665,0.004153,-0.044195,-22.756276
515,-0.213086,-0.208364,-0.204968,-0.138773,-0.150648,-0.147735,-0.093741,-0.113939,-0.114427,-0.183307,...,0.063785,-0.135240,-0.083925,-0.047553,-0.067133,-0.049542,-0.049665,0.004210,-0.044458,0.042586
516,-0.177821,-0.201289,-0.208907,-0.137745,-0.149337,-0.148886,-0.082596,-0.113162,-0.114529,0.255098,...,0.063758,-0.135596,-0.083949,-0.047553,-0.067135,-0.049542,-0.049665,0.004286,-0.044458,0.042586
517,-0.190247,-0.191969,-0.179230,-0.139127,-0.149164,-0.146973,-0.087570,-0.113040,-0.112485,0.094879,...,0.063840,10.405662,6.252507,-0.045956,-0.067135,-0.049542,-0.049665,0.004272,-0.044458,0.042586


In [1346]:
pca = PCA(n_components=40)

printcipalComponents = pca.fit_transform(scaled_X)

# principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2', '3'])

pca.explained_variance_ratio_

array([2.36113605e-01, 1.26576976e-01, 9.33887097e-02, 8.19759116e-02,
       6.70883091e-02, 5.26506438e-02, 4.81953475e-02, 4.53906045e-02,
       4.44703296e-02, 4.19684643e-02, 2.66923024e-02, 2.57750148e-02,
       1.88357776e-02, 1.79435576e-02, 1.36845365e-02, 9.90841449e-03,
       9.34980108e-03, 7.33601501e-03, 5.94043890e-03, 5.02653998e-03,
       3.48843843e-03, 2.98305010e-03, 2.25029857e-03, 2.10425977e-03,
       1.66860907e-03, 1.62237356e-03, 1.26929009e-03, 1.14729895e-03,
       9.40760454e-04, 5.64045917e-04, 5.42189571e-04, 4.29639324e-04,
       3.68601983e-04, 3.62296391e-04, 3.36224385e-04, 3.03714681e-04,
       2.47612336e-04, 2.33899532e-04, 1.58392846e-04, 1.30651376e-04])

In [1347]:
principalDF = pd.DataFrame(printcipalComponents)

In [1348]:
principalDF

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.400664,-0.453696,0.004906,-0.736854,0.615565,-0.272698,-0.118930,-0.114894,-0.200645,-0.060542,...,-0.093882,0.031576,-0.057904,0.116377,-0.044797,0.024535,0.092295,0.046914,0.018692,-0.010740
1,-0.592725,-0.526858,-0.101938,-0.358550,0.983993,-0.356003,0.076374,0.111932,-0.076422,0.079836,...,-0.069905,0.168392,-0.150173,0.138104,0.080143,-0.023392,0.262150,-0.050035,0.299181,0.153038
2,-0.372678,-0.476071,0.060673,-0.800718,0.549547,-0.245579,-0.118893,-0.189140,-0.242311,-0.099380,...,-0.031427,0.055514,0.011398,0.098955,-0.030848,-0.008947,0.130504,0.067204,-0.002702,0.022158
3,-0.454418,-0.486317,0.024862,-0.788759,0.524675,-0.199025,-0.103599,-0.240208,-0.313481,-0.097138,...,0.003223,0.091518,0.015158,0.101198,-0.036563,0.005769,0.096592,0.034715,0.042014,0.082841
4,9.785008,-0.287093,-0.349177,-0.667701,0.571715,-0.246048,-0.153433,-0.306156,-0.084215,-0.109861,...,-1.509335,-0.158529,0.049377,-0.524933,1.407380,-0.320453,-0.548393,-0.082969,-0.547121,1.099751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,-0.878889,-1.080363,-1.287322,9.894504,3.968890,-0.904218,-0.391131,-0.647615,-1.819286,46.571599,...,-0.009689,0.017708,-0.090158,-0.025778,-0.016630,0.006520,0.021264,-0.110534,-0.005267,0.003201
515,-0.580832,-0.307067,-0.116902,-0.220994,0.336420,-0.472035,-0.087360,0.038882,-0.130893,0.070284,...,0.022788,-0.072793,-0.144574,-0.135071,-0.027962,0.022684,-0.114365,0.010651,0.039331,-0.001338
516,-0.555736,-0.491754,-0.098946,-0.380969,0.583107,-0.191148,-0.223224,-0.029164,-0.182482,0.109315,...,0.021987,0.012478,0.086501,0.131698,-0.003350,0.049524,-0.042271,0.046034,-0.127331,-0.005203
517,-1.208301,-1.026844,-1.697781,12.175349,-21.979069,-1.835689,0.804959,0.752496,1.089392,-0.400979,...,0.234146,0.281773,-0.093728,-0.052065,0.003287,0.101493,-0.181243,0.030260,-0.053123,0.083106


In [1349]:
X = principalDF.values

y = data.iloc[:,-1]

In [1350]:
torch.manual_seed(100)

In [1351]:
# 데이터 전처리 (예시: 텐서로 변환)
x = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [1352]:
# 데이터 분할 (예시: 훈련 세트와 테스트 세트로 분할)
train_size = int(0.8 * len(x))  # 훈련 세트의 크기 설정
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [1377]:
# 모델 구성
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 4*input_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*input_size, 2*input_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(2*input_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.sigmoid(out)
        return out

input_size = x.shape[1]  
model = BinaryClassifier(input_size)


# 손실 함수와 옵티마이저 설정
criterion = nn.BCELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# optimizer = optim.RMSprop(model.parameters(), lr=0.01)



# 모델 학습
num_epochs = 9
batch_size = 64

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_acc = 0.0

    # 미니배치 학습
    for i in range(0, len(x_train), batch_size):
        batch_x, batch_y = x_train[i:i+batch_size], y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += accuracy_score(batch_y.detach().numpy(), (outputs > 0.5).float().detach().numpy())

    epoch_loss /= len(x_train) / batch_size
    epoch_acc /= len(x_train) / batch_size

    print(f'Epoch {epoch+1} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}')

# 모델 평가
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    predictions = (outputs > 0.5).float()
    test_acc = accuracy_score(y_test.detach().numpy(), predictions.detach().numpy())
    print(f'Test Accuracy: {test_acc:.4f}')

Epoch 1 - Loss: 0.9771 - Accuracy: 0.5826
Epoch 2 - Loss: 0.6165 - Accuracy: 0.8309
Epoch 3 - Loss: 0.5806 - Accuracy: 0.8402
Epoch 4 - Loss: 0.5666 - Accuracy: 0.8016
Epoch 5 - Loss: 0.5486 - Accuracy: 0.8766
Epoch 6 - Loss: 0.5399 - Accuracy: 0.8670
Epoch 7 - Loss: 0.5329 - Accuracy: 0.8742
Epoch 8 - Loss: 0.5257 - Accuracy: 0.8739
Epoch 9 - Loss: 0.5204 - Accuracy: 0.8982
Test Accuracy: 0.8846
